# 直线的点向式与一般式互换（CS/AI专项笔记·精研版）
## 前言
直线的点向式（对称式）与一般式互换是**3D空间几何的核心转化工具**，其本质是将“定点+方向向量”的直观几何表达（点向式）与“两平面交线”的代数通用表达（一般式）相互转换。在CS/AI领域，这种互换是3D数据处理的关键环节：自动驾驶的轨迹建模（点向式）需转化为一般式用于障碍物碰撞检测，3D场景的平面交线（一般式）需转化为点向式用于路径规划，计算机图形学的线条渲染（点向式）需基于一般式进行交点计算。本章以“定义→互换推导→步骤拆解→AI场景→代码实现→避坑指南”为脉络，系统拆解互换逻辑，结合可运行代码与3D可视化，适配Mac Jupyter环境，确保理论与工程落地无缝衔接。

## 1. 核心定义（两种方程的本质与AI场景映射）
### 1.1 点向式（对称式）方程
#### 1.1.1 严格数学定义
若直线 $L$ 过定点 $P_0(x_0,y_0,z_0)$，方向向量为 $\vec{s}=(l,m,n)$（$l,m,n$ 不同时为0，称为方向数），则点向式方程为：
$$\boxed{\frac{x - x_0}{l} = \frac{y - y_0}{m} = \frac{z - z_0}{n}}$$
- 特殊情况：若某方向数为0（如 $l=0$），方程等价于 $\begin{cases} x = x_0 \\ \frac{y - y_0}{m} = \frac{z - z_0}{n} \end{cases}$（直线垂直于x轴）。

#### 1.1.2 AI场景映射
| 核心要素 | 数学意义 | AI工程价值 | 场景示例 |
|----------|----------|------------|----------|
| 定点 $P_0$ | 直线的空间位置基准 | 确定轨迹起点、点云种子点 | 自动驾驶车辆起始位置、3D直线分割种子点 |
| 方向向量 $\vec{s}$ | 直线的延伸方向 | 控制运动方向、线条走向 | 机械臂运动方向、道路边缘线延伸趋势 |
| 方向数 $l,m,n$ | 方向向量的分量 | 量化方向特征，适配拟合算法 | 3D点云直线拟合的参数输出 |

### 1.2 一般式方程
#### 1.2.1 严格数学定义
若直线 $L$ 是两不平行平面 $\Pi_1: A_1x+B_1y+C_1z+D_1=0$ 与 $\Pi_2: A_2x+B_2y+C_2z+D_2=0$ 的交线，则一般式方程为两平面方程的联立：
$$\boxed{\begin{cases} A_1x + B_1y + C_1z + D_1 = 0 \\ A_2x + B_2y + C_2z + D_2 = 0 \end{cases}}$$
- 核心约束：两平面法向量 $\vec{n_1}=(A_1,B_1,C_1)$ 与 $\vec{n_2}=(A_2,B_2,C_2)$ 不共线（$A_1:B_1:C_1 \neq A_2:B_2:C_2$），否则直线不存在。

#### 1.2.2 AI场景映射
| 核心要素 | 数学意义 | AI工程价值 | 场景示例 |
|----------|----------|------------|----------|
| 平面 $\Pi_1,\Pi_2$ | 直线的约束条件 | 从场景平面中提取交线 | 3D场景中墙面与地面的交线、道路与绿化带的边界 |
| 法向量 $\vec{n_1},\vec{n_2}$ | 平面的朝向 | 推导直线方向向量（叉积） | 一般式转点向式的关键步骤 |
| 系数 $A_1,B_1,C_1,D_1$ | 平面的代数表达 | 紧凑存储，适配实时计算 | 自动驾驶障碍物边界的参数化存储 |

### 1.3 两种方程的核心关联（互换基础）
- 几何关联：直线的方向向量 $\vec{s}$ 同时垂直于两平面的法向量 $\vec{n_1}$ 和 $\vec{n_2}$，即 $\vec{s} = \vec{n_1} \times \vec{n_2}$（叉积）；
- 代数关联：点向式的参数关系可拆分为两个线性方程（对应一般式的两平面），一般式的两平面交线可通过法向量叉积得到方向向量，再通过解方程得到定点。

## 2. 互换推导与详细步骤（几何意义+代数运算）
### 2.1 点向式 → 一般式（核心：拆分参数关系为线性方程）
#### 2.1.1 推导逻辑
点向式的本质是“三个等式联立”（$\frac{x-x_0}{l} = \frac{y-y_0}{m} = \frac{z-z_0}{n} = t$），通过拆分任意两个等式，可得到两个独立的线性方程，即一般式的两平面方程。

#### 2.1.2 通用步骤（分情况讨论）
##### 情况1：方向数 $l,m,n$ 均不为0
1. 令点向式等于参数 $t$，得到参数式：
   $$\begin{cases} x = x_0 + lt \\ y = y_0 + mt \\ z = z_0 + nt \end{cases}$$
2. 拆分前两个等式消去 $t$：$\frac{x-x_0}{l} = \frac{y-y_0}{m} → m(x-x_0) - l(y-y_0) = 0 → mx - ly + (ly_0 - mx_0) = 0$（平面 $\Pi_1$）；
3. 拆分后两个等式消去 $t$：$\frac{y-y_0}{m} = \frac{z-z_0}{n} → n(y-y_0) - m(z-z_0) = 0 → ny - mz + (mz_0 - ny_0) = 0$（平面 $\Pi_2$）；
4. 联立两方程，得到一般式：
   $$\boxed{\begin{cases} mx - ly + (ly_0 - mx_0) = 0 \\ ny - mz + (mz_0 - ny_0) = 0 \end{cases}}$$

##### 情况2：某方向数为0（如 $l=0$，$m,n \neq 0$）
1. 点向式等价于 $\begin{cases} x = x_0 \\ \frac{y-y_0}{m} = \frac{z-z_0}{n} \end{cases}$；
2. 第一个方程直接作为平面 $\Pi_1: x - x_0 = 0$（法向量 $\vec{n_1}=(1,0,0)$）；
3. 拆分第二个等式得到平面 $\Pi_2: ny - mz + (mz_0 - ny_0) = 0$；
4. 一般式：
   $$\boxed{\begin{cases} x - x_0 = 0 \\ ny - mz + (mz_0 - ny_0) = 0 \end{cases}}$$

#### 2.1.3 几何意义
拆分得到的两个平面，分别垂直于 $xy$ 平面和 $yz$ 平面（或其他坐标平面组合），其交线恰好是原直线——本质是用两个正交约束平面“框定”直线的空间位置。

### 2.2 一般式 → 点向式（核心：求方向向量+定点）
#### 2.2.1 推导逻辑
一般式的两平面法向量叉积得到直线方向向量，再通过求解两平面方程得到直线上的一个定点，最终代入点向式公式。

#### 2.2.2 通用步骤（含验证）
1. **求方向向量 $\vec{s}$**：
   - 两平面法向量 $\vec{n_1}=(A_1,B_1,C_1)$，$\vec{n_2}=(A_2,B_2,C_2)$；
   - 直线方向向量 $\vec{s} = \vec{n_1} \times \vec{n_2}$（因 $\vec{s}$ 需同时垂直于 $\vec{n_1}$ 和 $\vec{n_2}$），计算式：
     $$\vec{s} = \begin{vmatrix} \vec{i} & \vec{j} & \vec{k} \\ A_1 & B_1 & C_1 \\ A_2 & B_2 & C_2 \end{vmatrix} = (B_1C_2 - B_2C_1, C_1A_2 - C_2A_1, A_1B_2 - A_2B_1)$$
   - 验证：若 $\vec{s} = \vec{0}$，则两平面平行，直线不存在。

2. **求直线上的定点 $P_0(x_0,y_0,z_0)$**：
   - 令某一坐标为常数（如 $z=0$，若直线不平行于 $z$ 轴），代入一般式，解二元一次方程组：
     $$\begin{cases} A_1x + B_1y + D_1 = 0 \\ A_2x + B_2y + D_2 = 0 \end{cases}$$
   - 若方程组无解，换另一坐标为常数（如 $y=0$），直至求出有效解；
   - 特殊情况：若直线平行于 $z$ 轴（$C_1=C_2=0$），则 $z$ 可任意取值，令 $x=x_0$（从第一个方程解出），$y=y_0$，$z$ 取任意值（如 $z=0$）。

3. **构造点向式**：
   - 若 $\vec{s}=(l,m,n)$，$P_0=(x_0,y_0,z_0)$，则点向式为：
     $$\boxed{\frac{x - x_0}{l} = \frac{y - y_0}{m} = \frac{z - z_0}{n}}$$
   - 处理方向数为0的情况：若 $l=0$，则分子为 $x - x_0 = 0$，其余项正常保留。

## 3. AI场景深度应用案例（互换的工程价值）
### 3.1 案例1：点向式→一般式（自动驾驶轨迹碰撞检测）
#### 问题背景
自动驾驶车辆的行驶轨迹用点向式表示（已知起点和方向），需转化为一般式，与障碍物的平面方程联立，判断轨迹是否与障碍物相交（碰撞检测）。

#### 工程流程
1. 轨迹建模：车辆起点 $P_0(0,0,0)$，行驶方向 $\vec{s}=(1,1,0)$（平面运动），点向式为 $\frac{x}{1} = \frac{y}{1} = \frac{z}{0}$；
2. 点向式转一般式：拆分得到 $\begin{cases} x - y = 0 \\ z = 0 \end{cases}$；
3. 碰撞检测：障碍物用平面 $\Pi: x + y - 10 = 0$ 表示，联立一般式与平面方程，求解交点 $(5,5,0)$，判断是否在轨迹范围内。

#### 代码实现（Mac Jupyter适配）
```python
import numpy as np

def point_direction_to_general(P0, s):
    """
    点向式转一般式
    :param P0: 定点 (x0,y0,z0)
    :param s: 方向向量 (l,m,n)
    :return: 一般式系数 ((A1,B1,C1,D1), (A2,B2,C2,D2))
    """
    x0, y0, z0 = P0
    l, m, n = s

    # 情况1：l != 0 且 m != 0
    if l != 0 and m != 0:
        # 第一个平面：mx - ly + (ly0 - mx0) = 0
        A1, B1, C1 = m, -l, 0
        D1 = l * y0 - m * x0
        # 第二个平面：根据n是否为0选择
        if n != 0:
            # 第二个平面：ny - mz + (mz0 - ny0) = 0
            A2, B2, C2 = 0, n, -m
            D2 = m * z0 - n * y0
        else:
            # n=0，第二个平面：z = z0 → 0x + 0y + 1z - z0 = 0
            A2, B2, C2 = 0, 0, 1
            D2 = -z0
    # 情况2：l=0（x=x0）
    elif l == 0 and m != 0 and n != 0:
        A1, B1, C1 = 1, 0, 0
        D1 = -x0
        A2, B2, C2 = 0, n, -m
        D2 = m * z0 - n * y0
    # 情况3：m=0（y=y0）
    elif m == 0 and l != 0 and n != 0:
        A1, B1, C1 = 0, 1, 0
        D1 = -y0
        A2, B2, C2 = n, 0, -l
        D2 = l * z0 - n * x0
    else:
        raise ValueError("方向向量不能为零向量，且至少两个方向数非零")

    return (A1,B1,C1,D1), (A2,B2,C2,D2)

# 自动驾驶轨迹：点向式转一般式
P0 = (0, 0, 0)
s = (1, 1, 0)
plane1, plane2 = point_direction_to_general(P0, s)
print(f"轨迹点向式：x/1 = y/1 = z/0")
print(f"转化为一般式：")
print(f"平面1：{plane1[0]}x + {plane1[1]}y + {plane1[2]}z + {plane1[3]} = 0")
print(f"平面2：{plane2[0]}x + {plane2[1]}y + {plane2[2]}z + {plane2[3]} = 0")

# 碰撞检测：与障碍物平面 x + y - 10 = 0 联立
A_obstacle, B_obstacle, C_obstacle, D_obstacle = 1, 1, 0, -10
# 联立一般式与障碍物平面方程
# 一般式：x - y = 0；z = 0；障碍物平面：x + y -10 =0
x = 5
y = 5
z = 0
print(f"\n轨迹与障碍物平面交点：({x},{y},{z})")
print(f"交点是否在轨迹上：{'是' if (x==y and z==0) else '否'}")
```

### 3.2 案例2：一般式→点向式（3D点云直线拟合后建模）
#### 问题背景
3D点云分割得到平面交线（一般式），需转化为点向式，用于后续轨迹规划（明确方向向量）和3D可视化（绘制直线）。

#### 代码实现（Mac Jupyter适配）
```python
def general_to_point_direction(plane1, plane2):
    """
    一般式转点向式
    :param plane1: 第一个平面 (A1,B1,C1,D1)
    :param plane2: 第二个平面 (A2,B2,C2,D2)
    :return: 定点 P0 (x0,y0,z0)，方向向量 s (l,m,n)
    """
    A1, B1, C1, D1 = plane1
    A2, B2, C2, D2 = plane2

    # 步骤1：计算方向向量 s = n1 × n2
    n1 = np.array([A1, B1, C1])
    n2 = np.array([A2, B2, C2])
    s = np.cross(n1, n2)
    l, m, n = s
    if np.linalg.norm(s) < 1e-10:
        raise ValueError("两平面平行，无公共直线")

    # 步骤2：求直线上的定点 P0（令z=0，解方程组）
    # 方程组：A1x + B1y = -D1；A2x + B2y = -D2
    try:
        # 构造系数矩阵和常数项
        coeff = np.array([[A1, B1], [A2, B2]])
        const = np.array([-D1, -D2])
        x0, y0 = np.linalg.solve(coeff, const)
        z0 = 0
    except np.linalg.LinAlgError:
        # z=0时方程组无解，令y=0
        coeff = np.array([[A1, C1], [A2, C2]])
        const = np.array([-D1, -D2])
        x0, z0 = np.linalg.solve(coeff, const)
        y0 = 0

    return (x0, y0, z0), (l, m, n)

# 模拟3D点云平面交线（一般式）
plane1 = (1, 1, 1, -6)  # x + y + z -6 = 0
plane2 = (2, -1, 1, -3) # 2x - y + z -3 = 0
P0, s = general_to_point_direction(plane1, plane2)
l, m, n = s

print(f"一般式：")
print(f"平面1：{plane1[0]}x + {plane1[1]}y + {plane1[2]}z + {plane1[3]} = 0")
print(f"平面2：{plane2[0]}x + {plane2[1]}y + {plane2[2]}z + {plane2[3]} = 0")
print(f"\n转化为点向式：")
print(f"定点 P0：({P0[0]:.4f}, {P0[1]:.4f}, {P0[2]:.4f})")
print(f"方向向量 s：({l:.4f}, {m:.4f}, {n:.4f})")
print(f"点向式方程：(x - {P0[0]:.4f})/{l:.4f} = (y - {P0[1]:.4f})/{m:.4f} = (z - {P0[2]:.4f})/{n:.4f}")

# 3D可视化直线（点向式）
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
# 生成直线点（t从-5到5）
t = np.linspace(-5, 5, 100)
x = P0[0] + l * t
y = P0[1] + m * t
z = P0[2] + n * t
# 绘制直线
ax.plot(x, y, z, color='red', linewidth=3, label='一般式转化后的直线')
# 绘制定点和方向向量
ax.scatter(P0[0], P0[1], P0[2], color='blue', s=100, marker='*', label='定点 P0')
ax.quiver(P0[0], P0[1], P0[2], l*2, m*2, n*2, color='green', linewidth=2, label='方向向量 s')

ax.set_xlabel('X轴')
ax.set_ylabel('Y轴')
ax.set_zlabel('Z轴')
ax.set_title('一般式转点向式：3D点云直线建模', fontsize=14)
ax.legend()
plt.show()
```

## 4. 常见误区与避坑指南（初学者+AI工程视角）
<html>
<table style="width:100%; border-collapse: collapse; margin: 16px 0; font-size: 14px;">
  <thead>
    <tr style="background-color: #f5f5f5;">
      <th style="padding: 12px; text-align: left; border: 1px solid #ddd; font-weight: 600;">易错点</th>
      <th style="padding: 12px; text-align: left; border: 1px solid #ddd; font-weight: 600;">错误示例</th>
      <th style="padding: 12px; text-align: left; border: 1px solid #ddd; font-weight: 600;">正确做法</th>
      <th style="padding: 12px; text-align: left; border: 1px solid #ddd; font-weight: 600;">AI工程影响</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td style="padding: 12px; border: 1px solid #ddd;">点向式转一般式时忽略方向数为0的情况</td>
      <td style="padding: 12px; border: 1px solid #ddd;">将点向式 $\frac{x-2}{0} = \frac{y+1}{2} = \frac{z-3}{-1}$ 错误拆分为 $\begin{cases} 2(x-2) - 0(y+1) = 0 \\ -1(y+1) - 2(z-3) = 0 \end{cases}$</td>
      <td style="padding: 12px; border: 1px solid #ddd;">方向数为0时，对应分子为常数方程（如 $x=2$），正确一般式为 $\begin{cases} x-2=0 \\ -y - 2z + 5 = 0 \end{cases}$</td>
      <td style="padding: 12px; border: 1px solid #ddd;">碰撞检测时方程无解，误判为无碰撞</td>
    </tr>
    <tr style="background-color: #fafafa;">
      <td style="padding: 12px; border: 1px solid #ddd;">一般式转点向式时方向向量计算错误</td>
      <td style="padding: 12px; border: 1px solid #ddd;">将法向量 $\vec{n_1}=(1,1,1)$ 与 $\vec{n_2}=(2,-1,1)$ 的叉积误算为 $(2,1,-3)$（正确应为 $(2,1,-3)$ 是正确结果，此处示例错误为 $(1,0,-1)$）</td>
      <td style="padding: 12px; border: 1px solid #ddd;">严格按照行列式展开叉积，或用numpy.cross()验证，确保 $\vec{s} \perp \vec{n_1}$ 且 $\vec{s} \perp \vec{n_2}$（点积为0）</td>
      <td style="padding: 12px; border: 1px solid #ddd;">轨迹方向错误，自动驾驶车辆偏离规划路径</td>
    </tr>
    <tr>
      <td style="padding: 12px; border: 1px solid #ddd;">定点求解时选择不当坐标导致无解</td>
      <td style="padding: 12px; border: 1px solid #ddd;">一般式 $\begin{cases} x + z = 0 \\ y + z = 0 \end{cases}$ 中，令 $x=1$ 求解，得到 $z=-1$，$y=1$，但误判为无解</td>
      <td style="padding: 12px; border: 1px solid #ddd;">选择坐标时优先令 $z=0$ 或 $y=0$，若方程组无解，换另一坐标，避免选择与直线平行的坐标轴</td>
      <td style="padding: 12px; border: 1px solid #ddd;">无法生成点向式，3D直线可视化失败</td>
    </tr>
    <tr style="background-color: #fafafa;">
      <td style="padding: 12px; border: 1px solid #ddd;">混淆3D与2D直线的一般式</td>
      <td style="padding: 12px; border: 1px solid #ddd;">将3D直线的一般式误写为单个线性方程（如 $x + y + z = 5$）</td>
      <td style="padding: 12px; border: 1px solid #ddd;">3D直线的一般式必须是两平面联立方程，单个线性方程表示平面</td>
      <td style="padding: 12px; border: 1px solid #ddd;">3D场景建模错误，将平面误判为直线</td>
    </tr>
    <tr>
      <td style="padding: 12px; border: 1px solid #ddd;">忽略一般式的两平面不平行条件</td>
      <td style="padding: 12px; border: 1px solid #ddd;">对平行平面 $\begin{cases} x + y + z = 1 \\ 2x + 2y + 2z = 3 \end{cases}$ 尝试转化为点向式</td>
      <td style="padding: 12px; border: 1px solid #ddd;">转化前先验证两平面法向量是否共线（叉积是否为零向量），共线则无直线</td>
      <td style="padding: 12px; border: 1px solid #ddd;">代码报错（除零错误），点云处理流程中断</td>
    </tr>
  </tbody>
</table>
</html>

## 5. 学习建议（CS/AI方向专属）
1. **锚定“方向向量与法向量的垂直关系”**：互换的核心是 $\vec{s} = \vec{n_1} \times \vec{n_2}$，学习时先理解该几何本质，再推导代数步骤，避免死记公式；
2. **分情况突破特殊案例**：方向数为0、平面方程无解等特殊情况是工程中高频场景，需单独练习，确保代码能处理所有边界条件；
3. **绑定AI场景记忆互换目的**：
   - 点向式→一般式：用于碰撞检测、交点计算（代数联立更高效）；
   - 一般式→点向式：用于轨迹规划、3D可视化（方向向量更直观）；
4. **强化代码自动化互换能力**：用numpy封装互换函数，包含异常处理（如平行平面、方向向量为零），适配工程中批量处理需求（如多段轨迹的方程转换）；
5. **衔接后续知识**：互换是直线与平面交点计算、3D点云直线聚类、机器人运动学正逆解的基础，建议同步学习“直线与平面的交点求解”，深化工程应用能力。

## 6. 自测问题（含详细解题过程）
### 自测题1：点向式→一般式（基础）
将直线的点向式方程 $\frac{x - 3}{2} = \frac{y + 1}{-1} = \frac{z - 4}{3}$ 转化为一般式方程。

#### 详细解题过程：
- 步骤1：令点向式等于 $t$，得到参数式：
  $$\begin{cases} x = 3 + 2t \\ y = -1 - t \\ z = 4 + 3t \end{cases}$$
- 步骤2：拆分前两个等式消去 $t$：
  $$\frac{x-3}{2} = \frac{y+1}{-1} → -(x-3) = 2(y+1) → -x + 3 = 2y + 2 → x + 2y - 1 = 0$$（平面 $\Pi_1$）；
- 步骤3：拆分后两个等式消去 $t$：
  $$\frac{y+1}{-1} = \frac{z-4}{3} → 3(y+1) = -(z-4) → 3y + 3 = -z + 4 → 3y + z - 1 = 0$$（平面 $\Pi_2$）；
- 步骤4：联立两平面方程，得到一般式：
  $$\boxed{\begin{cases} x + 2y - 1 = 0 \\ 3y + z - 1 = 0 \end{cases}}$$

### 自测题2：一般式→点向式（核心，含特殊情况）
将直线的一般式方程 $\begin{cases} x - z + 2 = 0 \\ y + 2z - 3 = 0 \end{cases}$ 转化为点向式方程。

#### 详细解题过程：
- 步骤1：计算方向向量 $\vec{s}$：
  - 两平面法向量 $\vec{n_1}=(1,0,-1)$，$\vec{n_2}=(0,1,2)$；
  - 叉积 $\vec{s} = \vec{n_1} \times \vec{n_2} = \begin{vmatrix} \vec{i} & \vec{j} & \vec{k} \\ 1 & 0 & -1 \\ 0 & 1 & 2 \end{vmatrix} = (0×2 - (-1)×1, -1×0 - 1×2, 1×1 - 0×0) = (1, -2, 1)$；
- 步骤2：求定点 $P0$（令 $z=0$）：
  - 代入一般式：$\begin{cases} x + 2 = 0 \\ y - 3 = 0 \end{cases} → x=-2$，$y=3$，$z=0$，即 $P0(-2,3,0)$；
- 步骤3：构造点向式方程：
  $$\boxed{\frac{x + 2}{1} = \frac{y - 3}{-2} = \frac{z}{1}}$$

### 自测题3：AI场景应用题（综合）
在自动驾驶轨迹规划中，车辆的初始轨迹用一般式 $\begin{cases} 2x - y = 0 \\ z - 1 = 0 \end{cases}$ 表示，需转化为点向式用于路径可视化和方向控制。
1. 转化为点向式方程；
2. 若车辆沿该轨迹行驶，当 $t=2$ 时（参数 $t$ 对应行驶距离），求车辆的位置坐标。

#### 详细解题过程：
#### 1. 一般式→点向式转化
- 步骤1：计算方向向量 $\vec{s}$：
  - 法向量 $\vec{n_1}=(2,-1,0)$，$\vec{n_2}=(0,0,1)$；
  - 叉积 $\vec{s} = \vec{n_1} \times \vec{n_2} = ( (-1)×1 - 0×0, 0×0 - 2×1, 2×0 - (-1)×0 ) = (-1, -2, 0)$；
- 步骤2：求定点 $P0$（令 $x=0$）：
  - 代入一般式：$\begin{cases} -y = 0 \\ z - 1 = 0 \end{cases} → y=0$，$z=1$，即 $P0(0,0,1)$；
- 步骤3：点向式方程：
  $$\boxed{\frac{x}{-1} = \frac{y}{-2} = \frac{z - 1}{0}}$$

#### 2. 求 $t=2$ 时的位置坐标
- 步骤1：将点向式转化为参数式：
  $$\begin{cases} x = 0 - 1×t \\ y = 0 - 2×t \\ z = 1 + 0×t = 1 \end{cases}$$
- 步骤2：代入 $t=2$：
  $$x = -2$，$y = -4$，$z=1$；
- 结果：车辆位置坐标为 $\boxed{(-2, -4, 1)}$。

## 总结
直线的点向式与一般式互换是CS/AI领域3D空间几何的核心转化技能，其本质是“直观几何表达”与“通用代数表达”的相互适配。点向式转一般式的核心是拆分参数关系为两线性方程，一般式转点向式的核心是通过法向量叉积求方向向量、解方程求定点。本章系统梳理了互换的推导逻辑、步骤拆解、AI场景应用与代码实现，重点突出了工程中常见的特殊情况与避坑指南。学习时需紧扣“几何本质→代数运算→工程实现”的逻辑链，绑定自动驾驶、3D点云处理等场景强化记忆，确保能灵活应用于轨迹规划、碰撞检测、3D可视化等核心工程任务。

需要我为你补充**直线与平面的交点计算（基于互换后的方程）** 或**多直线方程的批量互换（工程化优化）** 吗？